## フーリエ変換を用いたテキストのクラス分類器を実装する

In [1]:
import os
import pandas as pd
import numpy as np
from typing import Dict, List
import importlib
from pathlib import Path
import matplotlib.pyplot as plt
# import MeCab
from datasets import load_dataset
import pickle

C:\Users\zigza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BASE_PATH = Path.cwd().parent
DATA_PATH = BASE_PATH / "data"
MODEL_PATH = BASE_PATH / "model"
print(f"BASE_PATH: {BASE_PATH}")
print(f"DATA_PATH: {DATA_PATH}")
print(f"MODEL_PATH: {MODEL_PATH}")

BASE_PATH: c:\Users\zigza\OneDrive\ドキュメント\git\Fourier-TextClassifier
DATA_PATH: c:\Users\zigza\OneDrive\ドキュメント\git\Fourier-TextClassifier\data
MODEL_PATH: c:\Users\zigza\OneDrive\ドキュメント\git\Fourier-TextClassifier\model


In [3]:
import python.util as util

importlib.reload(util)
# ? logger読み込み
name = "15-Ensemble"
logger = util.set_logger()
# ? seed値固定
seed = 42
util.set_seed(seed)

2024-06-22 15:00:16,742 : python.util : INFO : 34 : Test_message


### DATASETS
今回は<a href="https://huggingface.co/datasets/stanfordnlp/imdb">Large Movie Review Dataset</a>を用いる．

In [7]:
#INFO: データ読み込み
#DATA_PATHが空のフォルダである場合は、データのダウンロードを行う
with util.timer("Read_data"):
    if os.listdir(DATA_PATH) == []:
        logger.info("No files in data folder")
        ds = load_dataset("stanfordnlp/imdb")
    else:
        logger.info("Files exist in data folder")
        with open(DATA_PATH / "train.pkl", "rb") as f:
            train = pickle.load(f)
        with open(DATA_PATH / "test.pkl", "rb") as f:
            test = pickle.load(f)
        ds = {"train": train, "test": test}

    #INFO: データの分割
    train = ds["train"]
    test = ds["test"]

    #INFO: データの保存
    with open(DATA_PATH / "train.pkl", "wb") as f:
        pickle.dump(train, f)
    with open(DATA_PATH / "test.pkl", "wb") as f:
        pickle.dump(test, f)

[Read_data] start
No files in data folder
[Read_data] done in 12.97 s
